In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
states = ['US-AL','US-AK','US-AZ','US-AR','US-CA','US-CO','US-CT', 'US-DE', 'US-FL', 'US-GA', 'US-HI',
          'US-ID', 'US-IL','US-IN','US-IA','US-KS','US-KY','US-LA','US-ME','US-MD','US-MA', 'US-MI','US-MN',
          'US-MS','US-MO','US-MT','US-NE','US-NV','US-NH','US-NJ','US-NM','US-NY','US-NC','US-ND','US-OH',
          'US-OK','US-OR','US-PA','US-RI','US-SC','US-SD','US-TN','US-TX','US-UT','US-VT','US-VA','US-WA',
          'US-WV','US-WI','US-WY','US-DC']

# for state in states:
#     print(f'{state} is my home')

In [3]:
store_data = []

overpass_url = "http://overpass-api.de/api/interpreter"

skips_by_state = {}

def increment_skip(state, skips_by_state):
    if state in skips_by_state:
        skips_by_state[state] += 1
    else:
        skips_by_state[state] = 1
    return skips_by_state

for state in states:
    overpass_query = f"[out:json][timeout:120];"\
                     f"area['ISO3166-2'='{state}'][admin_level=4];"\
                     f"(node['shop'='supermarket'](area);"\
                     f" way['shop'='supermarket'](area);"\
                     f" rel['shop'='supermarket'](area);"\
                     f");"\
                     f"out center;"

    response = requests.get(overpass_url, 
                        params={'data': overpass_query})
    
    data = response.json()
    
    stores = data['elements']

    for store in stores:
        loop_list = {}  
        if store['type'] == 'node':
            if store['lat'] and store['lon'] not in loop_list:
                try:
                    loop_list['name'] = store['tags']['name']  
                    loop_list['lat'] = store['lat']
                    loop_list['lon'] = store['lon']
                    store_data.append(loop_list)
                except KeyError:
                    skips_by_state = increment_skip(state, skips_by_state)
                    # print('Missing Field')
        elif 'center' in store:
            if store["lat"] and store["lon"] not in loop_list:
                try:
                    loop_list['name'] = store['tags']['name']  
                    loop_list['lat'] = store['lat']
                    loop_list['lon'] = store['lon']
                    store_data.append(loop_list)
                except KeyError:
                    skips_by_state = increment_skip(state, skips_by_state)

print(len(store_data))
# store_data

13351


In [12]:
skips_by_state
# skips = list(skips_by_state)
# skip_df = pd.DataFrame(skips)
# skip_df

{'US-AL': 4,
 'US-AK': 3,
 'US-AZ': 13,
 'US-AR': 3,
 'US-CA': 47,
 'US-CO': 5,
 'US-CT': 4,
 'US-FL': 24,
 'US-GA': 10,
 'US-ID': 7,
 'US-IL': 11,
 'US-IN': 13,
 'US-IA': 6,
 'US-KS': 14,
 'US-KY': 4,
 'US-LA': 8,
 'US-ME': 6,
 'US-MD': 8,
 'US-MA': 6,
 'US-MI': 15,
 'US-MN': 8,
 'US-MS': 3,
 'US-MO': 7,
 'US-MT': 4,
 'US-NE': 12,
 'US-NV': 3,
 'US-NH': 2,
 'US-NJ': 15,
 'US-NM': 5,
 'US-NY': 40,
 'US-NC': 15,
 'US-OH': 12,
 'US-OK': 4,
 'US-OR': 2,
 'US-PA': 20,
 'US-SC': 6,
 'US-SD': 3,
 'US-TN': 4,
 'US-TX': 28,
 'US-UT': 2,
 'US-VT': 2,
 'US-VA': 17,
 'US-WA': 10,
 'US-WV': 3,
 'US-WI': 10,
 'US-DC': 1}

In [20]:
us_stores_df = pd.DataFrame(store_data)
us_stores_df = us_stores_df[['name', 'lat', 'lon']]
# us_stores_df
grouped_by_name_df = us_stores_df.groupby(['name']).count().


lat    84
lon    84
Name: Target, dtype: int64

In [22]:
us_stores_df.to_csv('us_supermarkets.csv')